In [0]:
import json
import re
import cni_connectors.adls_gen1_connector as connector
import pyspark.sql.functions as f
import crawler.functions as cf
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, LongType, DoubleType

In [0]:
var_adls_uri = connector.adls_gen1_connect(spark, dbutils, scope="adls_gen2", dynamic_overwrite="dynamic")

fornecedor_dev
dev-execution

In [0]:
#Parametros de entrado do ADF
var_file = {'namespace':'API_InGuru',
'file_folder':'inguro_acidente_trab_trab_judicializados',
'extension':'csv',
'column_delimiter':',',
'encoding':'UTF-8',
'null_value':''}

#Caminho de Dev
var_dls = {
    "folders":{"landing":"/tmp/dev/uld",
    "error":"/tmp/dev/err",
    "staging":"/tmp/dev/stg",
    "log":"/tmp/dev/log",
    "raw":"/tmp/dev/raw",
    "trusted":"/tmp/dev/trs",
    "business":"/tmp/dev/biz",
    "landing":"/tmp/dev/uld"          }}
#Parametros do ADF (Nativo)
var_adf = {
    "adf_factory_name": "cnibigdatafactory",
    "adf_pipeline_name": "org_raw_acidente_trab_judicializados",
    "adf_pipeline_run_id": "60ee3485-4a56-4ad1-99ae-666666666",
    "adf_trigger_id": "62bee9e9-acbb-49cc-80f2-666666666",
    "adf_trigger_name": "62bee9e9-acbb-49cc-80f2-66666666",
    "adf_trigger_time": "2020-06-08T01:42:41.5507749Z",
    "adf_trigger_type": "PipelineActivity"
}

In [0]:
#Acesso ao json do CMD
lnd = var_dls['folders']['landing']
raw = var_dls['folders']['raw']

In [0]:
var_source = "{lnd}/{namespace}/{file_folder}/".format(lnd=lnd, namespace=var_file['namespace'], file_folder=var_file['file_folder'])
var_source

Out[5]: '/tmp/dev/uld/API_InGuru/inguro_acidente_trab_trab_judicializados/'

In [0]:
#Origem e destino
source_lnd = "{adl_path}{lnd}/{namespace}/{file_folder}".format(adl_path=var_adls_uri, lnd=lnd, namespace=var_file["namespace"],file_folder=var_file["file_folder"])
source_lnd

Out[6]: '[REDACTED]/tmp/dev/uld/API_InGuru/inguro_acidente_trab_trab_judicializados'

In [0]:
azure_path = 'abfss://datalake@cnibigdatadlsgen2.dfs.core.windows.net'
outputh_path = '{uri}/uds/uniepro/API_InGuru/inguro_vig_acidente_trab'.format(uri=azure_path)

def API_INGURU(url, per_page, start_date, end_date):
    headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36',
    'accept':'application/json',
    'authorization': '6f64d6a047fad369d35c3806f8f8e0560475075a432585973f91caae35b4ad74'
    } 

    params = {
        'per_page': per_page,
        'start_date': start_date,
        'end_date': end_date,
        'sort': '1'
    }
    response = requests.get(url, headers=headers, params=params)
    return response.json()

In [0]:
def divide_data(data_inicial, data_final):
    formato_data = '%d/%m/%Y %H:%M'

    data_inicial = datetime.strptime(data_inicial, formato_data)
    data_final = datetime.strptime(data_final, formato_data)
    intervalo = data_final - data_inicial
    data_final = data_inicial + intervalo // 2
    data_final = data_final.replace(hour=23, minute=59, second=59) # define o final como o último segundo do minuto
    data_final = data_final.strftime(formato_data)
    data_inicial = data_inicial.strftime(formato_data)
    return (data_inicial, data_final)

In [0]:
def load_big_df(path):
  schema = StructType([
    StructField("id", LongType(), nullable=True),
    StructField("domain", StringType(), nullable=True),
    StructField("title", StringType(), nullable=True),
    StructField("subtitle", StringType(), nullable=True),
    StructField("author", StringType(), nullable=True),
    StructField("content", StringType(), nullable=True),
    StructField("url", StringType(), nullable=True),
    StructField("source", StringType(), nullable=True),
    StructField("source_country", StringType(), nullable=True),
    StructField("source_state", StringType(), nullable=True),
    StructField("crawled_date", StringType(), nullable=True),
    StructField("published_date", StringType(), nullable=True),
    StructField("dh_insertion_raw", StringType(), nullable=True)
  ])

  df = spark.read.schema(schema) \
             .option("mergeSchema", "true") \
             .option("timestampFormat", "INT96") \
             .parquet(path) \
             .withColumn("dh_insertion_raw", f.col("dh_insertion_raw").cast("timestamp"))\
             .withColumn("crawled_date", f.col("crawled_date").cast("timestamp"))
  
  return df

In [0]:
def get_news(data_inicial, data_final, id):
    url = 'https://app.inguru.me/api/v1/taxonomies/nodes/news/'+str(id)

    array_dfs = []

    getted_news = 0
    
    total_news = API_INGURU(url, 1, data_inicial, data_final)['pagination']['total']
    
    if not total_news:
      dados = {
        'author': np.nan,
        'content': np.nan,
        'crawled_date': np.nan,
        'domain': np.nan,
        'id': np.nan,
        'published_date': np.nan,
        'source': np.nan,
        'source_country': np.nan,
        'source_state': np.nan,
        'subtitle': np.nan,
        'title': np.nan,
        'url': np.nan,
        'dh_insertion_raw': np.nan
      }
      fail_df = pd.DataFrame(dados, index=[0])
      array_dfs.append(fail_df)
      parcial_data_inicial = parcial_data_final
      parcial_data_final = data_final
    else:
      
      print('Total de notícias: ', total_news)

      parcial_data_inicial = data_inicial
      parcial_data_final = data_final

      while getted_news < total_news:

          parcial_news = total_news

          while parcial_news > 10000:
#               print('Parcial de notícias: ', parcial_news)
              parcial_data_inicial, parcial_data_final = divide_data(parcial_data_inicial, parcial_data_final)
              parcial_news = API_INGURU(url, 1, parcial_data_inicial, parcial_data_final)['pagination']['total']

#           print("fazendo requisicao")
#           print('Data inicial: ', parcial_data_inicial)
#           print('Data final: ', parcial_data_final)
          news = API_INGURU(url, parcial_news, parcial_data_inicial, parcial_data_final)['data']

          #CRIA UM DATAFRAME COM AS NOTICIAS
          noticias = pd.DataFrame(news)
          #ADICIONA NUM ARRAY DE DATAFRAME
          array_dfs.append(noticias)

          #corrige as datas

          parcial_data_inicial = parcial_data_final
          parcial_data_final = data_final

          getted_news += parcial_news
          

    return pd.concat(array_dfs)

### Ingestão de anos anteriores

Teve de ser criado pois a API não carrega todos os anos por alguma limitação.

PRECISA SER RODADO APENAS NA PRIMEIRA VEZ

In [0]:
def insere_news(data_inicial, data_final, id, path):
  url = 'https://app.inguru.me/api/v1/taxonomies/nodes/news/'+str(id)

  array_dfs = []

  getted_news = 0
    
  total_news = API_INGURU(url, 1, data_inicial, data_final)['pagination']['total']
    
  if total_news:
      
    print('Total de notícias: ', total_news)

    parcial_data_inicial = data_inicial
    parcial_data_final = data_final

    while getted_news < total_news:

      parcial_news = total_news

      while parcial_news > 10000:
#         print('Parcial de notícias: ', parcial_news)
        parcial_data_inicial, parcial_data_final = divide_data(parcial_data_inicial, parcial_data_final)
        parcial_news = API_INGURU(url, 1, parcial_data_inicial, parcial_data_final)['pagination']['total']

#       print("fazendo requisicao")
#       print('Data inicial: ', parcial_data_inicial)
#       print('Data final: ', parcial_data_final)
      news = API_INGURU(url, parcial_news, parcial_data_inicial, parcial_data_final)['data']

      #CRIA UM DATAFRAME COM AS NOTICIAS
      noticias = pd.DataFrame(news)

      #corrige as datas
      parcial_data_inicial = parcial_data_final
      parcial_data_final = data_final

      getted_news += parcial_news

      dh_insertion_raw = var_adf["adf_trigger_time"].split(".")[0]

      schema = StructType([
        StructField("id", LongType(), nullable=True),
        StructField("domain", StringType(), nullable=True),
        StructField("title", StringType(), nullable=True),
        StructField("subtitle", StringType(), nullable=True),
        StructField("author", StringType(), nullable=True),
        StructField("content", StringType(), nullable=True),
        StructField("url", StringType(), nullable=True),
        StructField("source", StringType(), nullable=True),
        StructField("source_country", StringType(), nullable=True),
        StructField("source_state", StringType(), nullable=True),
        StructField("crawled_date", StringType(), nullable=True),
        StructField("published_date", StringType(), nullable=True)
      ])

      saveDF = spark.createDataFrame(noticias, schema)
      saveDF = saveDF.withColumn("dh_insertion_raw", f.lit(dh_insertion_raw).cast("string"))
      saveDF.write.parquet(path, "append")
      print("Total:", total_news)
      print("Inseridas:", getted_news)
    
  else:
    print("Sem dados")


In [0]:
graph_ids = []
graph_names = []
graph_dfs = []

nodeID = '928' #Declaração do ID

url_nodeID = 'https://app.inguru.me/api/v1/taxonomies/nodes/'+str(nodeID)
IDs = API_INGURU(url = url_nodeID, per_page=1, start_date='', end_date='')['data'][0]
graph_ids.append(IDs['id'])
graph_names.append(IDs['name'].replace(" ", "_"))

#Passa por cada "filho" no nó principal, caso não tenha nenhum, nada acontece.
for node_children in IDs['children']:
  graph_ids.append(node_children['id'])
  graph_names.append(node_children['name'].replace(" ", "_"))

data_inicial = '01/01/2015 00:00'
data_final = datetime.now().strftime("%d/%m/%Y %H:%M")

for i, id in enumerate(graph_ids):
  path = source_lnd + "_" + str(graph_names[i])
  insere_news(data_inicial, data_final, id, path)


Total de notícias: 673827
Total: 673827
Inseridas: 456
Total: 673827
Inseridas: 8547
Total: 673827
Inseridas: 17478
Total: 673827
Inseridas: 22107
Total: 673827
Inseridas: 23172
Total: 673827
Inseridas: 23721
Total: 673827
Inseridas: 25754
Total: 673827
Inseridas: 35135
Total: 673827
Inseridas: 45074
Total: 673827
Inseridas: 54874
Total: 673827
Inseridas: 64387
Total: 673827
Inseridas: 69061
Total: 673827
Inseridas: 74819
Total: 673827
Inseridas: 84543
Total: 673827
Inseridas: 93838
Total: 673827
Inseridas: 100292
Total: 673827
Inseridas: 106552
Total: 673827
Inseridas: 112139
Total: 673827
Inseridas: 120762
Total: 673827
Inseridas: 129437
Total: 673827
Inseridas: 138818
Total: 673827
Inseridas: 145045
Total: 673827
Inseridas: 153552
Total: 673827
Inseridas: 160371
Total: 673827
Inseridas: 170196
Total: 673827
Inseridas: 179508
Total: 673827
Inseridas: 189199
Total: 673827
Inseridas: 197753
Total: 673827
Inseridas: 206578
Total: 673827
Inseridas: 213267
Total: 673827
Inseridas: 221606
Total: 673827
Inseridas: 230665
Total: 673827
Inseridas: 237744
Total: 673827
Inseridas: 243035
Total: 673827
Inseridas: 251302
Total: 673827
Inseridas: 257203
Total: 673827
Inseridas: 263916
Total: 673827
Inseridas: 270118
Total: 673827
Inseridas: 276598
Total: 673827
Inseridas: 283106
Total: 673827
Inseridas: 288634
Total: 673827
Inseridas: 296016
Total: 673827
Inseridas: 302732
Total: 673827
Inseridas: 309778
Total: 673827
Inseridas: 315912
Total: 673827
Inseridas: 322494
Total: 673827
Inseridas: 329631
Total: 673827
Inseridas: 335567
Total: 673827
Inseridas: 341438
Total: 673827
Inseridas: 351369
Total: 673827
Inseridas: 360704
Total: 673827
Inseridas: 370110
Total: 673827
Inseridas: 379727
Total: 673827
Inseridas: 388036
Total: 673827
Inseridas: 394880
Total: 673827
Inseridas: 402087
Total: 673827
Inseridas: 409409
Total: 673827
Inseridas: 415736
Total: 673827
Inseridas: 423504
Total: 673827
Inseridas: 429506
Total: 673827
Inseridas: 436942
Total: 673827
Inseridas: 442411
Total: 673827
Inseridas: 448306
Total: 673827
Inseridas: 453052
Total: 673827
Inseridas: 458645
Total: 673827
Inseridas: 464949
Total: 673827
Inseridas: 471267
Total: 673827
Inseridas: 477659
Total: 673827
Inseridas: 485653
Total: 673827
Inseridas: 493846
Total: 673827
Inseridas: 502057
Total: 673827
Inseridas: 508616
Total: 673827
Inseridas: 516492
Total: 673827
Inseridas: 523499
Total: 673827
Inseridas: 528444
Total: 673827
Inseridas: 535666
Total: 673827
Inseridas: 540775
Total: 673827
Inseridas: 548979
Total: 673827
Inseridas: 552722
Total: 673827
Inseridas: 560015
Total: 673827
Inseridas: 566643
Total: 673827
Inseridas: 574355
Total: 673827
Inseridas: 583981
Total: 673827
Inseridas: 590935
Total: 673827
Inseridas: 597036
Total: 673827
Inseridas: 606933
Total: 673827
Inseridas: 614752
Total: 673827
Inseridas: 623694
Total: 673827
Inseridas: 630978
Total: 673827
Inseridas: 637085
Total: 673827
Inseridas: 646315
Total: 673827
Inseridas: 653328
Total: 673827
Inseridas: 660695
Total: 673827
Inseridas: 668393
Total: 673827
Inseridas: 673839
Total de notícias: 5149
Total: 5149
Inseridas: 5149

In [0]:
insere_news(data_inicial, data_final, graph_ids[1], path)

Total de notícias: 4224
Total: 4224
Inseridas: 4224

In [0]:
# #ZERAR A BASE
# #Deve-se rodar a célula 16 antes de executar esta

# dh_insertion_raw = var_adf["adf_trigger_time"].split(".")[0]

# data = {
#   'id': 0,
#   'domain': 'Não informado',
#   'title': 'Não informado',
#   'subtitle': 'Não informado',
#   'author': 'Não informado',
#   'content': 'Não informado',
#   'url': 'Não informado',
#   'source': 'Não informado',
#   'source_country': 'Não informado',
#   'source_state': 'Não informado',
#   'crawled_date': '1900-01-01 00:00:00',
#   'published_date': '1900-01-01 00:00:00',
#   'dh_insertion_raw': '2020-06-08T01:42:41.000+0000'
# }
# schema = StructType([
#   StructField("id", LongType(), nullable=True),
#   StructField("domain", StringType(), nullable=True),
#   StructField("title", StringType(), nullable=True),
#   StructField("subtitle", StringType(), nullable=True),
#   StructField("author", StringType(), nullable=True),
#   StructField("content", StringType(), nullable=True),
#   StructField("url", StringType(), nullable=True),
#   StructField("source", StringType(), nullable=True),
#   StructField("source_country", StringType(), nullable=True),
#   StructField("source_state", StringType(), nullable=True),
#   StructField("crawled_date", StringType(), nullable=True),
#   StructField("published_date", StringType(), nullable=True),
#   StructField("dh_insertion_raw", StringType(), nullable=True)
#   ])

# data['id'] = int(data['id'])


# spark_dfff = spark.createDataFrame([tuple(data.values())], schema=schema)
# for i, df in enumerate(graph_dfs):
#   path = source_lnd + "_" + str(graph_names[i])
#   spark_dfff.write.parquet(path, mode="overwrite")
  